<a href="https://colab.research.google.com/github/paridhika/DDL/blob/main/ML_split_Cost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# According to PyTorch

In [ ]:
from torchvision.models import resnet50
import torch
from torchsummary import summary

In [ ]:
summary(resnet50(), (3, 224, 224))

# According to TF

In [ ]:
import tensorflow as tf

In [ ]:
model = tf.keras.applications.resnet.ResNet50(weights=None)
model.summary();

# Get layers with tensors

In [ ]:
from tensorflow.python.ops.math_ops import sub
# import tensorflow as tf
# model = tf.keras.applications.resnet.ResNet50(weights='imagenet');
# model.summary();
import csv  
layer_count = 0
my_map = {}
for i, layer in enumerate(model.layers):
    if "add" not in layer.name:
        layer_count += 1
        # print("Layer {}: {} ({} parameters)".format(layer_count, layer.name, layer.count_params()))
        my_map[layer.name] = layer.count_params()
  
print(len(my_map))

map_list = list(my_map.items())
map_list.reverse()
with open("tensor_layers.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(map_list)

161


# Get layers with parameters

In [ ]:
from tensorflow.python.ops.math_ops import sub
# import tensorflow as tf
# model = tf.keras.applications.resnet.ResNet50(weights='imagenet');
# model.summary();
import csv  
layer_count = 0
my_map = {}
for i, layer in enumerate(model.layers):
    if len(layer.weights) > 0:
      if "add" not in layer.name:
        layer_count += 1
        # print("Layer {}: {} ({} parameters)".format(layer_count, layer.name, layer.count_params()))
        my_map[layer.name] = layer.count_params()
  
print(len(my_map))

map_list = list(my_map.items())
map_list.reverse()
with open("parameters_layers.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(map_list)

107


# Compute burst size

In [ ]:
from tensorflow.python.ops.math_ops import sub
# import tensorflow as tf
# model = tf.keras.applications.resnet.ResNet50(weights='imagenet');
# model.summary();
import csv  
layer_count = 0
substring = ""
my_map = {}


for i, layer in enumerate(model.layers):
    if len(layer.weights) > 0:
      if "_" in layer.name:
        layer_count += 1
        # print("Layer {}: {} ({} parameters)".format(layer_count, layer.name, layer.count_params()))
        substring = layer.name[:layer.name.rindex("_")]
        if substring in my_map:
          my_map[substring] = my_map[substring] + layer.count_params()
        else:
          my_map[substring] = layer.count_params()
      else:
        my_map[layer.name] = layer.count_params()
  
print(len(my_map))
my_map["Layer Name"] = "Theoritical"
map_list = list(my_map.items())
map_list.reverse()
with open("my_map.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(map_list)





54


**Get table of comparison between theoritical and experimental burst sizes**

In [ ]:
import csv
import pandas as pd
def generate_latex_table(filename):
    with open(filename, newline='') as csvfile:
        reader = csv.reader(csvfile)
        header = next(reader)
        table = []
        for row in reader:
            table.append(row)
        
    table_string = "\\begin{tabular}{|" + "c|"*len(header) + "}\n\\hline\n"
    table_string += " & ".join(header) + "\\\\\n\\hline\n"
    for row in table:
        table_string += " & ".join(row) + "\\\\\n\\hline\n"
    table_string += "\\end{tabular}"
    
    return table_string

numbers = [2049839.5,1048383.5,2359135.5,1048447.5,1048367.5,2359167.5,1048367.5,1048367.5,2096959.5,2359311.5,524079.5,262015.5,589647.5,261983.5,261951.5,589695.5,261951.5,262015.5,589615.5,261935.5,261935.5,589663.5,262015.5,262527.5,589615.5,261919.5,261935.5,524143.5,589599.5,130943.5,65359.5,147231.5,65375.5,65551.5,147247.5,65343.5,65391.5,147231.5,65327.5,65551.5,130879.5,147231.5,0,0,0,0,0,0,0,0,0,0,0,0]

rounded_values = [int(round(number)) for number in numbers]

# Load the existing CSV file into a pandas dataframe
df = pd.read_csv("my_map.csv")

# Create a new column in the dataframe with the rounded integer values
df["Experimental"] = rounded_values

# Save the updated dataframe to the existing CSV file
df.to_csv("my_map.csv", index=False)

filename = 'my_map.csv'
index_col = [i for i in range(1, 55)] #index column
index_col = index_col[::-1]
with open(filename, 'r') as file:
    reader = csv.reader(file)
    header = next(reader) #get header
    header.insert(0, 'Burst Count') #insert "Index" header in the first position
    data = [row for row in reader] #get data

#create new data with index column
new_data = []
for i, row in enumerate(data):
    row.insert(0, index_col[i])
    new_data.append(row)

#write data to a new file
with open('new_file.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(new_data)



In [ ]:
import csv

# Open the CSV file for reading
with open('new_file.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    data = [row for row in reader]

# Round off all floating point values to integers
for i, row in enumerate(data):
    data[i] = [int(float(cell)) if '.' in cell else cell for cell in row]

# Write the updated data back to the same CSV file
with open('final.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(data)



In [ ]:
table_string = generate_latex_table("new_file.csv")
with open('latex_bursts.tex', 'w') as f:
        f.write(table_string)

**Compute number of parameters by formula**

In [ ]:
import tensorflow as tf

# Instantiate the ResNet50 model
model =  tf.keras.applications.resnet.ResNet50(weights=None) 

# Define a dictionary to store the number of operations for each layer
ops_dict = {}
param_dict = {}
# Define a function to compute the number of operations for a Conv2D layer
def compute_ops_conv2d(layer):
    kernel_size = layer.kernel_size[0]
    in_channels = layer.input_shape[-1]
    out_channels = layer.output_shape[-1]
    ops = kernel_size**2 * in_channels * out_channels
    return ops

# Define a function to compute the number of operations for a MaxPooling2D layer
def compute_ops_maxpool2d(layer):
    pool_size = layer.pool_size[0]
    in_channels = layer.input_shape[-1]
    ops = pool_size**2 * in_channels
    return ops

# Define a function to compute the number of operations for a ReLU layer
def compute_ops_relu(layer):
    num_elements = tf.reduce_prod(layer.input_shape[1:])
    ops = num_elements
    return ops

# Define a function to compute the number of operations for the output layer
def compute_ops_output(layer):
    num_input_channels = layer.input_shape[-1]
    num_output_channels = layer.output_shape[-1]
    fc_ops = (num_input_channels * num_output_channels) + num_output_channels
    if type(layer.activation) == tf.keras.activations.softmax or type(layer.activation) ==  tf.keras.activations.sigmoid:
        num_elements = tf.reduce_prod(layer.output_shape[1:])
        act_ops = num_elements
    else:
        act_ops = 0
    ops = fc_ops + act_ops
    return ops

# Compute the number of operations for each layer in the model
for layer in model.layers:
    if isinstance(layer, tf.keras.layers.Conv2D):
        ops = compute_ops_conv2d(layer)
    elif isinstance(layer, tf.keras.layers.MaxPooling2D):
        ops = compute_ops_maxpool2d(layer)
    elif isinstance(layer, tf.keras.layers.Activation) and type(layer.activation) == tf.keras.activations.relu:
        ops = compute_ops_relu(layer)
    elif isinstance(layer, tf.keras.layers.Dense):
        ops = compute_ops_output(layer)
    elif isinstance(layer, tf.keras.layers.BatchNormalization):
        num_channels = layer.output_shape[-1]
        ops = 4 * num_channels
    else:
        ops = 0
    ops_dict[layer.name] = ops
    param_dict[layer.name] = layer.count_params()
# Print the number of operations for each layer in the model
for layer_name, ops in ops_dict.items():
    print(layer_name, ops, param_dict[layer_name])


**Make plot for comparison**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the csv file into a pandas DataFrame
df = pd.read_csv('my_map.csv')

# Extract the two columns you need
col1 = df['Theoritical']
col2 = df['Experimental']

# Plot the histogram
plt.plot(df.iloc[:43, 1]/1000000, color='blue', label='Theoritical')
plt.plot(df.iloc[:43, 2]/1000000, color='red', label='Experimental')

# Add a legend and labels for the x and y axes
plt.legend()
plt.xlabel('Count')
plt.ylabel('Data (MB)')

plt.show()

# Compute computation time

In [ ]:
import tensorflow as tf

# Instantiate the ResNet50 model
model =  tf.keras.applications.resnet.ResNet50(weights=None) 

# Print the input and output shape for each convolutional layer in the model
for layer in model.layers:
    if isinstance(layer, tf.keras.layers.Conv2D):
      print(layer.name, layer.input_shape, layer.output_shape,  layer.kernel_size, layer.filters)




In [10]:
from keras.api._v2.keras.layers import Activation
import tensorflow as tf
import numpy as np

# Instantiate the ResNet50 model
# model =  tf.keras.applications.resnet.ResNet50(weights=None) 
# model = tf.keras.applications.vgg16.VGG16(weights=None) 
# model =  tf.keras.applications.resnet.ResNet152(weights=None) 
model =  tf.keras.applications.InceptionResNetV2(weights=None) 
# model.summary()
# Define a dictionary to store the number of operations for each layer
ops_dict = {}

# Define a function to compute the number of operations for a Conv2D layer
def compute_ops_conv2d(layer):
    kernel_size = layer.kernel_size[0]
    in_channels = layer.input_shape[-1]
    out_channels = layer.output_shape[-1]
    input_height, input_width = layer.input_shape[1:3]
    output_height, output_width = layer.output_shape[1:3]
    ops = kernel_size**2 * in_channels * output_height * output_width * out_channels
    return ops

# Define a function to compute the number of operations for a MaxPooling2D layer
def compute_ops_maxpool2d(layer):
    pool_size = layer.pool_size[0]
    in_channels = layer.input_shape[-1]
    input_height, input_width = layer.input_shape[1:3]
    output_height, output_width = layer.output_shape[1:3]
    ops = pool_size**2 * in_channels * output_height * output_width
    return ops

# Define a function to compute the number of operations for a BatchNormalization layer
def compute_ops_batchnorm(layer):
    # Get input shape
    input_shape = layer.input_shape[1:]
    in_channels = input_shape[-1]

    # Number of operations per element (2 for mean and variance)
    num_ops = 2 * in_channels

    return num_ops

# Define a function to compute the number of operations for a ReLU layer
def compute_ops_relu(layer):
    # Get output shape
    # output_shape = layer.output_shape[1:]
    # num_elements = np.prod(output_shape)
    input_shape = layer.input_shape[1:]
    # Number of operations per element
    num_ops = 1
    for dim in input_shape:
        num_ops *= dim

    return num_ops

    # Total number of operations
    # total_ops = num_elements * num_ops

    # Add operations for global average pooling
    if input_shape[1] != output_shape[1] or input_shape[2] != output_shape[2]:
        pool_size = (input_shape[1] - output_shape[1] + 1, input_shape[2] - output_shape[2] + 1)
        num_pool_ops = np.prod(pool_size) * output_shape[-1]
        total_ops += num_pool_ops

    return total_ops

# Define a function to compute the number of operations for the output layer
def compute_ops_output(layer):
    num_input_channels = layer.input_shape[-1]
    num_output_channels = layer.output_shape[-1]
    fc_ops = (num_input_channels * num_output_channels) + num_output_channels
    if type(layer.activation) == tf.keras.activations.softmax or type(layer.activation) == tf.keras.activations.sigmoid:
        num_elements = tf.reduce_prod(layer.output_shape[1:])
        act_ops = num_elements
    else:
        act_ops = 0
    ops = fc_ops + act_ops
    return ops

# Compute the number of operations for each layer in the model
for layer in model.layers:
    if isinstance(layer, tf.keras.layers.Conv2D):
        ops = compute_ops_conv2d(layer)
    elif isinstance(layer, tf.keras.layers.MaxPooling2D):
        ops = compute_ops_maxpool2d(layer)
    elif isinstance(layer, tf.keras.layers.BatchNormalization):
        ops = compute_ops_batchnorm(layer)
    elif isinstance(layer, tf.keras.layers.Activation):
        ops = compute_ops_relu(layer)
    elif isinstance(layer, tf.keras.layers.Dense):
        ops = compute_ops_output(layer)
    else:
        ops = 0
    ops_dict[layer.name] = tuple([ops,layer.input_shape, layer.output_shape])

# Print the number of operations for each layer in the model
sum = 0;
batch_size = 32

i = 0
for layer_name, (ops, inp, out) in ops_dict.items():
    sum = sum + ops
    # print(layer_name, ops, inp, out)

    if i == 0:
      i = i+10
      (n,l,h,w) = inp[0]
      params = l*h*w * 4

sum = sum/1000000000 #GFLOP
GFLOP = sum * batch_size
print(f'input image : {params}')
params = params * batch_size / 1000000
print(f'GFLOP : {GFLOP}')

print(f'Total input images : {params} MB')

# print(f'Parameters Memory : 102 MB')

input image : 1072812
GFLOP : 643.197383936
Total input images : 34.329984 MB


In [8]:
import tensorflow as tf

# Define a dictionary to store the number of operations for each layer
ops_dict = {}

# Define a function to compute the number of operations for a Conv2D layer
def compute_ops_conv2d(layer):
    kernel_size = layer.kernel_size[0]
    in_channels = layer.input_shape[-1]
    out_channels = layer.output_shape[-1]
    input_height, input_width = layer.input_shape[1:3]
    output_height, output_width = layer.output_shape[1:3]
    ops = kernel_size**2 * in_channels * output_height * output_width * out_channels
    return ops

# Define a function to compute the number of operations for a MaxPooling2D layer
def compute_ops_maxpool2d(layer):
    pool_size = layer.pool_size[0]
    in_channels = layer.input_shape[-1]
    input_height, input_width = layer.input_shape[1:3]
    output_height, output_width = layer.output_shape[1:3]
    ops = pool_size**2 * in_channels * output_height * output_width
    return ops

# Define a function to compute the number of operations for a BatchNormalization layer
def compute_ops_batchnorm(layer):
    # Get input shape
    input_shape = layer.input_shape[1:]
    in_channels = input_shape[-1]

    # Number of operations per element (2 for mean and variance)
    num_ops = 2 * in_channels

    return num_ops

# Define a function to compute the number of operations for a ReLU layer
def compute_ops_relu(layer):
    # Get output shape
    # output_shape = layer.output_shape[1:]
    # num_elements = np.prod(output_shape)
    input_shape = layer.input_shape[1:]
    # Number of operations per element
    num_ops = 1
    for dim in input_shape:
        num_ops *= dim

    return num_ops

    # Total number of operations
    # total_ops = num_elements * num_ops

    # Add operations for global average pooling
    if input_shape[1] != output_shape[1] or input_shape[2] != output_shape[2]:
        pool_size = (input_shape[1] - output_shape[1] + 1, input_shape[2] - output_shape[2] + 1)
        num_pool_ops = np.prod(pool_size) * output_shape[-1]
        total_ops += num_pool_ops

    return total_ops

# Define a function to compute the number of operations for the output layer
def compute_ops_output(layer):
    num_input_channels = layer.input_shape[-1]
    num_output_channels = layer.output_shape[-1]
    fc_ops = (num_input_channels * num_output_channels) + num_output_channels
    if type(layer.activation) == tf.keras.activations.softmax or type(layer.activation) == tf.keras.activations.sigmoid:
        num_elements = tf.reduce_prod(layer.output_shape[1:])
        act_ops = num_elements
    else:
        act_ops = 0
    ops = fc_ops + act_ops
    return ops

# Instantiate the ResNet152 model
model = tf.keras.applications.ResNet152(weights=None)
# model.summary()

for layer in model.layers:
    if isinstance(layer, tf.keras.layers.Conv2D):
        ops = compute_ops_conv2d(layer)
    elif isinstance(layer, tf.keras.layers.MaxPooling2D):
        ops = compute_ops_maxpool2d(layer)
    elif isinstance(layer, tf.keras.layers.BatchNormalization):
        ops = compute_ops_batchnorm(layer)
    elif isinstance(layer, tf.keras.layers.Activation):
        ops = compute_ops_relu(layer)
    elif isinstance(layer, tf.keras.layers.Dense):
        ops = compute_ops_output(layer)
    else:
        ops = 0
    ops_dict[layer.name] = tuple([ops,layer.input_shape, layer.output_shape])

# Print the number of operations for each layer in the model
sum = 0;
batch_size = 32

i = 0
for layer_name, (ops, inp, out) in ops_dict.items():
    sum = sum + ops
    # print(layer_name, ops, inp, out)

    if i == 0:
      i = i+10
      (n,l,h,w) = inp[0]
      params = l*h*w * 4

sum = sum/1000000000 #GFLOP
GFLOP = sum * batch_size
print(f'input image : {params}')
params = params * batch_size / 1000000
print(f'GFLOP : {GFLOP}')

print(f'Total input images : {params} MB')

# print(f'Parameters Memory : 102 MB')


Model: "resnet152"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_5[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                          

In [7]:
#  Edge : Model	 i9-11900K;
#         Cores/Threads	8/16
#         Base Clock	3.5 GHz
#         Boost Clock	5.3 GHz
#         Cache	  16 MB
#         TDP	  125W
#         GFLOPs  925.6

# Raspberry Pi 4 
# GFLOPs 24 
# Memory 1 to 8 GB


Edge = 24
CE = 0

# Near the Edge : Model Intel Xeon E5-2666 v3	
# c4.16xlarge	vCPUs 64	GFLOPs 2144.0 	$1.75/
# c4.8xlarge	vCPUs 36	Memory 60 GiB	EBS only	On Demand $1.59/hr	Reserved $1.01/hr	
# c4.large	vCPUs 2	Memory 3.75 GiB	EBS only	On Demand $0.10/hr	Reserved $0.06/hr	GFLOPs 67.2

NTE = 2144
CNTE = 1.01/3600 # cost per second

# Cloud : inf1.24xlarge	8x AWS Inferentia chips with 16 Neuron Cores each	
#  Memory 192 (GiB)	TFLOPs	42.0
# inf1.24xlarge	vCPUs 96	Network b/w	100	EBS b/w 19	$4.721/Hr
# inf1.xlarge	vCPUs 4	Memory 8 GiB	EBS only	On Demand $0.23/hr	Reserved $0.14/hr GFLOPs = 2250
# inf2.48xlarge	vCPUs 192	Memory 768 GiB	EBS only	On Demand $12.98/hr	Reserved $8.18/hr GFLOPs = 8,400,000

Cloud = 2250
CC = 0.14/3600  # cost per second


# cost model from Bell Network charges $0.3078 for transferring 1GB over its OC3 link -> 150 Mbps
TCost = 307.8 # per MB
print(f'Cost for transmission : ${params/TCost}')

Cost for transmission : $0.062597738791423


In [10]:
# Bandwidth = 1 Gbps 
# Transmission time from E to NTE
BWENTE = 1000
TENTE = params/BWENTE

print(f'Transmission Time from Edge to Near the Edge : {TENTE} sec')

print(f'Propagation Delay from Edge to Near the Edge : 0.01 sec')
# Bandwidth = 20 Mbps
# Transmission time from E to Cloud
BWEC = 20
TEC = params/BWEC

print(f'Transmission Time from Edge to Cloud : {TEC} sec')


Transmission Time from Edge to Near the Edge : 0.019267584 sec
Propagation Delay from Edge to Near the Edge : 0.01 sec
Transmission Time from Edge to Cloud : 0.9633792 sec


In [7]:
TE = GFLOP / Edge
TNTE = GFLOP / NTE
TC = GFLOP / Cloud

print(f'Time for excecution at Edge : {TE} secs')
print(f'Time for excecution at NTE : {TNTE} secs')

print(f'Transmission Time from Edge to Near the Edge : {TENTE} sec')

print(f'Total time at Near the Edge : {TNTE + TENTE} sec')

print(f'Time for excecution at Cloud : {TC} secs')

print(f'Transmission Time from Edge to Cloud : {TEC} sec')

print(f'Total time at Cloud : {TEC + TC} sec')

MCE = TE * CE
MCNTE = TNTE * CNTE 
MCC = TC * CC

print(f'Cost for excecution at Edge : ${MCE}')
print(f'Cost for excecution at NTE : ${MCNTE}')
print(f'Cost for excecution at Cloud : ${MCC}')

Time for excecution at Edge : 15.045832501333335 secs
Time for excecution at NTE : 0.16842349814925375 secs
Transmission Time from Edge to Near the Edge : 0.9633792 sec
Total time at Near the Edge : 1.1318026981492537 sec
Time for excecution at Cloud : 0.16048888001422223 secs
Transmission Time from Edge to Cloud : 0.019267584 sec
Total time at Cloud : 0.17975646401422224 sec
Cost for excecution at Edge : $0.0
Cost for excecution at NTE : $4.725214809187397e-05
Cost for excecution at Cloud : $6.241234222775309e-06


In [ ]:
#  Memory required by ResNet152 = 242 MB (parameter) + 200 KB ( for intermediate activations)
Mem = 242 # MB

In [ ]:
!pip install tensorboard


In [ ]:
# %load_ext tensorboard
# !kill 24824
%tensorboard --logdir logs/train

# %reload_ext tensorboard


# Understanding Model Summary

In [ ]:
import tensorflow as tf
model = tf.keras.applications.resnet.ResNet50(weights='imagenet');
# model.summary();
len(model.layers)
trainableLayerNameList = [layer.name for layer in model.layers if layer.trainable_variables]
print(len(trainableLayerNameList));

In [ ]:
trainable = 0

# iterate over the layers in the model
for layer in model.layers:
    # check if the layer has trainable parameters
    if layer.trainable_variables:
        # if the layer has trainable parameters, increment the counter
        trainable += 1

# print the number of layers involved in the AllReduce operation
print(f'Number of layers with trainable parameters: {trainable}')

Number of layers with trainable parameters: 107


In [ ]:
allreduce_layers = 0;
for layer in model.layers:
    # check if the layer has trainable parameters
    if not layer.trainable_variables:
        # if the layer has trainable parameters, increment the counter
        if "add" in layer.name:
          allreduce_layers += 1
          
# print the number of layers involved in the AllReduce operation
print(f'Number of layers not involved in AllReduce: {allreduce_layers}')
print(f'Number of layers involved in AllReduce: {len(model.layers) - allreduce_layers}')

In [ ]:
# import tensorflow as tf
resnet101 = tf.keras.applications.resnet.ResNet101(weights='imagenet')
len(resnet101.layers)
trainableLayerNameList = [layer.name for layer in resnet101.layers if layer.trainable_variables]
print(len(trainableLayerNameList));
allreduce_layers = 0;
for layer in resnet101.layers:
    # check if the layer has trainable parameters
    if not layer.trainable_variables:
        if "add" in layer.name:
          allreduce_layers += 1
          # print(layer.name);
print(f'Number of layers not involved in AllReduce: {allreduce_layers}')

In [ ]:
vgg16 = tf.keras.applications.vgg16.VGG16(weights=None)
vgg16.summary();
print(f'Number of layers : {len(vgg16.layers)}')
trainableLayerNameList = [layer.name for layer in vgg16.layers if layer.trainable_variables]
# print(len(trainableLayerNameList));
allreduce_layers = 0;
for layer in vgg16.layers:
    # check if the layer has trainable parameters
    if not layer.trainable_variables:
        if "add" in layer.name:
          allreduce_layers += 1
          # print(layer.name);
print(f'Number of layers not involved in AllReduce: {allreduce_layers}')

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
import re
conv2d = 0;
for layer in vgg16.layers:
  if "_conv" in layer.name:
    conv2d += 1;
print(conv2d);

13


In [ ]:
print(trainableLayerNameList)